# Visualize trained network using Grad-CAM

In [1]:
import os
os.chdir("..")
print(os.getcwd())
from main import *

c:\Users\User\Desktop\Kevin\Partial-DANN


c:\Users\User\miniconda3\envs\uxnet3d\lib\site-packages\monai\utils\deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)


## Hyperparameters

In [2]:
class Args:
    # Data
    dataset = "smat"
    modality = "ct+mr"
    partially_labelled = True
    holdout_ratio = 0.1
    mode = "train"
    
    # Module
    module = "dann" 
    pretrained = None
    batch_size = 1
    loss = "tal"
    lr = 0.001
    optim = "AdamW"
    max_iter = 10000
    eval_step = 1000
    
    # Development
    deterministic = False
    alpha = False
    beta = False
    
    # Efficiency
    cache_rate = 0.1
    num_workers = 2

## Read data & Load Module from checkpoint
The code may be obsolete since it may not be updated along development. Thus, copy the code from the `main.py` if any error occurs.

In [3]:
config = configparser.ConfigParser()
config.read("./config.ini")
args = Args()

## Parameters
# Data
root = "C:\\Users\\User\\Desktop\\smat"
output = config["path"]["output"]
debug = config["path"]["debug"]
dataset = args.dataset
modality = args.modality  # {"ct", "mr", "ct+mr"}
partially_labelled = args.partially_labelled  # {True, False}
holdout_ratio = args.holdout_ratio
mode = args.mode  # {"train", "test"}

# Module
module_name = args.module  # {"segmentation", "dann"}
pretrained = args.pretrained
batch_size = args.batch_size
loss = args.loss  # {"dice2", "tal"}
lr = args.lr
optim = args.optim
max_iter = args.max_iter
eval_step = args.eval_step

# Development
deterministic = args.deterministic
alpha = args.alpha
beta = args.beta

# Efficiency
cache_rate = args.cache_rate
num_workers = args.num_workers

## Configurations: data_info and mod_init
data_info = datasets[dataset]
mod_init = modules[module_name]["initializer"]

In [4]:
print("root dir:", root)
print("checkpoint dir:", output)
print("\ndataset info:")
display(data_info)

root dir: C:\Users\User\Desktop\smat
checkpoint dir: ./checkpoints

dataset info:


{'name': 'SMAT',
 'class': medaset.lake.SMATDataset,
 'train_transforms': None,
 'val_transforms': None,
 'fg': {'ct': [3], 'mr': [1, 2]},
 'bg': {'ct': {1: 0, 2: 0}, 'mr': {3: 0}}}

In [5]:
train_dataset, val_dataset, test_dataset = get_datasets(
    data_info=data_info,
    modality=modality,
    holdout_ratio=holdout_ratio,
    partially_labelled=partially_labelled,
    root_dir=root,
    cache_rate=cache_rate,
    num_workers=num_workers,
    dev=alpha,
)
train_dataloader, val_dataloader, test_dataloader = mod_init.init_dataloaders(
    train_dataset, val_dataset, test_dataset, batch_size, alpha
)

c:\users\user\desktop\kevin\medaset\src\medaset\lake.py:115: UserWarning: Some images are removed due to the lack of associated label: ['SMAT_CT_010', 'SMAT_CT_011', 'SMAT_CT_016', 'SMAT_CT_040', 'SMAT_CT_047', 'SMAT_CT_049', 'SMAT_CT_055', 'SMAT_CT_095', 'SMAT_CT_123', 'SMAT_CT_135', 'SMAT_CT_139']
  warnings.warn(


** Dataset = SMAT
** Modality = ct+mr
** Validation split ratio = 0.1
** Partially Labelled = True
** Foreground =
  - ct: [3] 
  - mr: [1, 2]


Loading dataset: 100%|██████████| 1/1 [00:00<00:00, 1001.51it/s]

** # of Training data = {'ct': 101, 'mr': 89}
** # of Validation data = {'ct': 11, 'mr': 9}
** # of Testing data = {'ct': 14, 'mr': 11}


In [ ]:
module = mod_init.init_module(
    out_channels=data_info["num_classes"],
    loss=loss,
    optim=optim,
    lr=lr,
    data_info=data_info,
    modality=modality,
    partially_labelled=partially_labelled,
    device=device,
)
if pretrained:
    print("** Pretrained checkpoint =", pretrained)
    module.load(pretrained)
module = module.to(device)
module = module.eval()
iterator = iter(test_dataloader[0])